In [2]:
import tkinter as tk
from tkinter import messagebox
import random

# Função para verificar o status atual do jogo (vitória, empate ou continuação)
def verificar_status(tabuleiro):
    # Verifica vitória nas linhas
    for linha in tabuleiro:
        if linha[0] == linha[1] == linha[2] != ' ':
            return linha[0]

    # Verifica vitória nas colunas
    for coluna in range(3):
        if tabuleiro[0][coluna] == tabuleiro[1][coluna] == tabuleiro[2][coluna] != ' ':
            return tabuleiro[0][coluna]

    # Verifica vitória nas diagonais
    if tabuleiro[0][0] == tabuleiro[1][1] == tabuleiro[2][2] != ' ':
        return tabuleiro[0][0]
    if tabuleiro[0][2] == tabuleiro[1][1] == tabuleiro[2][0] != ' ':
        return tabuleiro[0][2]

    # Verifica empate
    if all(linha.count(' ') == 0 for linha in tabuleiro):
        return 'Empate'

    # O jogo ainda está em andamento
    return None

# Função para avaliar o tabuleiro e atribuir uma pontuação
def avaliar(tabuleiro):
    resultado = verificar_status(tabuleiro)
    if resultado == 'X':
        return 1
    elif resultado == 'O':
        return -1
    elif resultado == 'Empate':
        return 0

    # O jogo ainda não terminou
    return None

# Função Minimax com poda alfa-beta para determinar a melhor jogada para o computador
def minimax(tabuleiro, profundidade, alpha, beta, maximizando):
    resultado = verificar_status(tabuleiro)

    # Se o jogo terminou ou a profundidade máxima foi atingida, retorna a avaliação
    if resultado is not None or profundidade == 0:
        return avaliar(tabuleiro)

    if maximizando:
        melhor_valor = float('-inf')
        for i in range(3):
            for j in range(3):
                if tabuleiro[i][j] == ' ':
                    tabuleiro[i][j] = 'X'
                    valor = minimax(tabuleiro, profundidade - 1, alpha, beta, False)
                    tabuleiro[i][j] = ' '
                    melhor_valor = max(melhor_valor, valor)
                    alpha = max(alpha, melhor_valor)
                    if beta <= alpha:
                        break
        return melhor_valor
    else:
        melhor_valor = float('inf')
        for i in range(3):
            for j in range(3):
                if tabuleiro[i][j] == ' ':
                    tabuleiro[i][j] = 'O'
                    valor = minimax(tabuleiro, profundidade - 1, alpha, beta, True)
                    tabuleiro[i][j] = ' '
                    if valor is not None:
                        melhor_valor = min(melhor_valor, valor)
                        beta = min(beta, melhor_valor)
                        if beta <= alpha:
                            break
        return melhor_valor

# Função para encontrar a melhor jogada para o computador usando Minimax com poda alfa-beta
def encontrar_melhor_jogada(tabuleiro):
    melhor_valor = float('-inf')
    melhor_jogada = None
    for i in range(3):
        for j in range(3):
            if tabuleiro[i][j] == ' ':
                tabuleiro[i][j] = 'X'
                valor = minimax(tabuleiro, 3, float('-inf'), float('inf'), False)
                tabuleiro[i][j] = ' '
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_jogada = (i, j)
    return melhor_jogada

# Função chamada quando o jogador humano clica em um quadrado
def quadrado_clicado(i, j):
    if tabuleiro[i][j] == ' ' and not jogo_acabou:
        tabuleiro[i][j] = 'O'
        atualizar_tabuleiro()

        if verificar_vitoria(tabuleiro, 'O'):
            messagebox.showinfo("Jogo da Velha", "Você venceu!")
            reiniciar_jogo()
        elif verificar_status(tabuleiro) == 'Empate':
            messagebox.showinfo("Jogo da Velha", "O jogo empatou!")
            reiniciar_jogo()
        else:
            jogada_computador()
            if verificar_vitoria(tabuleiro, 'X'):
                messagebox.showinfo("Jogo da Velha", "O computador venceu!")
                reiniciar_jogo()
            elif verificar_status(tabuleiro) == 'Empate':
                messagebox.showinfo("Jogo da Velha", "O jogo empatou!")
                reiniciar_jogo()

# Função que simula a jogada do computador
def jogada_computador():
    melhor_jogada = encontrar_melhor_jogada(tabuleiro)
    if melhor_jogada:
        i, j = melhor_jogada
        tabuleiro[i][j] = 'X'
        atualizar_tabuleiro()

# Função para reiniciar o jogo
def reiniciar_jogo():
    global tabuleiro, jogo_acabou
    tabuleiro = [[' ' for _ in range(3)] for _ in range(3)]
    jogo_acabou = False
    atualizar_tabuleiro()

# Função para verificar a vitória de um jogador
def verificar_vitoria(tabuleiro, jogador):
    for linha in tabuleiro:
        if all(celula == jogador for celula in linha):
            return True

    for coluna in range(3):
        if all(tabuleiro[linha][coluna] == jogador for linha in range(3)):
            return True

    if all(tabuleiro[i][i] == jogador for i in range(3)) or all(tabuleiro[i][2 - i] == jogador for i in range(3)):
        return True

    return False

# Função para atualizar a interface gráfica do tabuleiro
def atualizar_tabuleiro():
    for i in range(3):
        for j in range(3):
            quadrados[i][j].config(text=tabuleiro[i][j])
    label_jogador.config(text="Você é 'O'. Computador é 'X'")

# Criação da janela principal
janela = tk.Tk()
janela.title("Jogo da Velha")

# Inicialização do tabuleiro e variável para verificar se o jogo acabou
tabuleiro = [[' ' for _ in range(3)] for _ in range(3)]
jogo_acabou = False

# Rótulo para exibir o jogador atual
label_jogador = tk.Label(janela, text="Você é 'O'. Computador é 'X'")
label_jogador.grid(row=0, column=0, columnspan=3)

# Criação da matriz de quadrados no tabuleiro
quadrados = []
for i in range(3):
    linha = []
    for j in range(3):
        quadrado = tk.Button(janela, text='', width=10, height=3, command=lambda i=i, j=j: quadrado_clicado(i, j))
        quadrado.grid(row=i + 1, column=j)
        linha.append(quadrado)
    quadrados.append(linha)

# Botão para reiniciar o jogo
btn_reiniciar = tk.Button(janela, text="Reiniciar Jogo", command=reiniciar_jogo)
btn_reiniciar.grid(row=4, column=0, columnspan=3)

# Iniciar o jogo com a jogada do computador
jogada_computador()

# Início do loop da interface gráfica
janela.mainloop()


In [1]:
import tkinter as tk
from tkinter import messagebox
import random

# Função para verificar o status atual do jogo (vitória, empate ou continuação)
def verificar_status(tabuleiro):
    # Verifica vitória nas linhas
    for linha in tabuleiro:
        if linha[0] == linha[1] == linha[2] != ' ':
            return linha[0]

    # Verifica vitória nas colunas
    for coluna in range(3):
        if tabuleiro[0][coluna] == tabuleiro[1][coluna] == tabuleiro[2][coluna] != ' ':
            return tabuleiro[0][coluna]

    # Verifica vitória nas diagonais
    if tabuleiro[0][0] == tabuleiro[1][1] == tabuleiro[2][2] != ' ':
        return tabuleiro[0][0]
    if tabuleiro[0][2] == tabuleiro[1][1] == tabuleiro[2][0] != ' ':
        return tabuleiro[0][2]

    # Verifica empate
    if all(linha.count(' ') == 0 for linha in tabuleiro):
        return 'Empate'

    # O jogo ainda está em andamento
    return None

# Função para avaliar o tabuleiro e atribuir uma pontuação
def avaliar(tabuleiro):
    resultado = verificar_status(tabuleiro)
    if resultado == 'X':
        return 1
    elif resultado == 'O':
        return -1
    elif resultado == 'Empate':
        return 0

    # O jogo ainda não terminou
    return None

# Função Minimax com poda alfa-beta para determinar a melhor jogada para o computador
def minimax(tabuleiro, profundidade, alpha, beta, maximizando):
    resultado = verificar_status(tabuleiro)

    # Se o jogo terminou ou a profundidade máxima foi atingida, retorna a avaliação
    if resultado is not None or profundidade == 0:
        return avaliar(tabuleiro)

    if maximizando:
        melhor_valor = float('-inf')
        for i in range(3):
            for j in range(3):
                if tabuleiro[i][j] == ' ':
                    tabuleiro[i][j] = 'X'
                    valor = minimax(tabuleiro, profundidade - 1, alpha, beta, False)
                    tabuleiro[i][j] = ' '
                    melhor_valor = max(melhor_valor, valor)
                    alpha = max(alpha, melhor_valor)
                    if beta <= alpha:
                        break
        return melhor_valor
    else:
        melhor_valor = float('inf')
        for i in range(3):
            for j in range(3):
                if tabuleiro[i][j] == ' ':
                    tabuleiro[i][j] = 'O'
                    valor = minimax(tabuleiro, profundidade - 1, alpha, beta, True)
                    tabuleiro[i][j] = ' '
                    if valor is not None:
                        melhor_valor = min(melhor_valor, valor)
                        beta = min(beta, melhor_valor)
                        if beta <= alpha:
                            break
        return melhor_valor

# Função para encontrar a melhor jogada para o computador usando Minimax com poda alfa-beta
def encontrar_melhor_jogada(tabuleiro):
    melhor_valor = float('-inf')
    melhor_jogada = None

    # Primeiro, avalie as jogadas do jogador humano
    for i in range(3):
        for j in range(3):
            if tabuleiro[i][j] == ' ':
                tabuleiro[i][j] = 'O'
                if verificar_vitoria(tabuleiro, 'O'):
                    tabuleiro[i][j] = ' '
                    return i, j
                tabuleiro[i][j] = ' '

    # Se o jogador humano não estiver prestes a vencer, avalie as jogadas do computador
    for i in range(3):
        for j in range(3):
            if tabuleiro[i][j] == ' ':
                tabuleiro[i][j] = 'X'
                valor = minimax(tabuleiro, 3, float('-inf'), float('inf'), False)
                tabuleiro[i][j] = ' '
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_jogada = (i, j)

    return melhor_jogada

# Função chamada quando o jogador humano clica em um quadrado
def quadrado_clicado(i, j):
    if tabuleiro[i][j] == ' ' and not jogo_acabou:
        tabuleiro[i][j] = 'O'
        atualizar_tabuleiro()

        if verificar_vitoria(tabuleiro, 'O'):
            messagebox.showinfo("Jogo da Velha", "Você venceu!")
            reiniciar_jogo()
        elif verificar_status(tabuleiro) == 'Empate':
            messagebox.showinfo("Jogo da Velha", "O jogo empatou!")
            reiniciar_jogo()
        else:
            jogada_computador()
            if verificar_vitoria(tabuleiro, 'X'):
                messagebox.showinfo("Jogo da Velha", "O computador venceu!")
                reiniciar_jogo()
            elif verificar_status(tabuleiro) == 'Empate':
                messagebox.showinfo("Jogo da Velha", "O jogo empatou!")
                reiniciar_jogo()

# Função que simula a jogada do computador
def jogada_computador():
    melhor_jogada = encontrar_melhor_jogada(tabuleiro)
    if melhor_jogada:
        i, j = melhor_jogada
        tabuleiro[i][j] = 'X'
        atualizar_tabuleiro()

# Função para reiniciar o jogo
def reiniciar_jogo():
    global tabuleiro, jogo_acabou
    tabuleiro = [[' ' for _ in range(3)] for _ in range(3)]
    jogo_acabou = False
    atualizar_tabuleiro()

# Função para verificar a vitória de um jogador
def verificar_vitoria(tabuleiro, jogador):
    for linha in tabuleiro:
        if all(celula == jogador for celula in linha):
            return True

    for coluna in range(3):
        if all(tabuleiro[linha][coluna] == jogador for linha in range(3)):
            return True

    if all(tabuleiro[i][i] == jogador for i in range(3)) or all(tabuleiro[i][2 - i] == jogador for i in range(3)):
        return True

    return False

# Função para atualizar a interface gráfica do tabuleiro
def atualizar_tabuleiro():
    for i in range(3):
        for j in range(3):
            quadrados[i][j].config(text=tabuleiro[i][j])
    label_jogador.config(text="Você é 'O'. Computador é 'X'")

# Criação da janela principal
janela = tk.Tk()
janela.title("Jogo da Velha")

# Inicialização do tabuleiro e variável para verificar se o jogo acabou
tabuleiro = [[' ' for _ in range(3)] for _ in range(3)]
jogo_acabou = False

# Rótulo para exibir o jogador atual
label_jogador = tk.Label(janela, text="Você é 'O'. Computador é 'X'")
label_jogador.grid(row=0, column=0, columnspan=3)

# Criação da matriz de quadrados no tabuleiro
quadrados = []
for i in range(3):
    linha = []
    for j in range(3):
        quadrado = tk.Button(janela, text='', width=10, height=3, command=lambda i=i, j=j: quadrado_clicado(i, j))
        quadrado.grid(row=i + 1, column=j)
        linha.append(quadrado)
    quadrados.append(linha)

# Botão para reiniciar o jogo
btn_reiniciar = tk.Button(janela, text="Reiniciar Jogo", command=reiniciar_jogo)
btn_reiniciar.grid(row=4, column=0, columnspan=3)

# Iniciar o jogo com a jogada do computador
jogada_computador()

# Início do loop da interface gráfica
janela.mainloop()
